In [11]:
import os
import glob
import cv2
from ultralytics import YOLO
import numpy as np

# -----------------------------
# Load Models
# -----------------------------
# Replace with the correct paths to your model weights.
plate_detector = YOLO("./yolo12_detection_best.pt")  # Detection model
plate_reader   = YOLO("./yolo12_recognition_best.pt")  # Recognition model

# -----------------------------
# Character Mapping (Reversed)
# -----------------------------
# Provided mapping from training (char -> int)
char_to_idx = {
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 
    'D': 10, 'S': 11, 'الف': 12, 'ب': 13, 'ت': 14, 'تشریفات': 15, 'ث': 16, 'ج': 17, 
    'د': 18, 'ز': 19, 'س': 20, 'ش': 21, 'ص': 22, 'ط': 23, 'ظ': 24, 'ع': 25, 'ف': 26, 
    'ق': 27, 'ل': 28, 'م': 29, 'ن': 30, 'ه': 31, 'ه\u200d': 32, 'و': 33, 'پ': 34, 
    'ژ (معلولین و جانبازان)': 35, 'ک': 36, 'گ': 37, 'ی': 38
}
# Reverse the mapping to get index -> character
idx_to_char = {v: k for k, v in char_to_idx.items()}

# -----------------------------
# Recognition Helper Function
# -----------------------------
def extract_plate_text(read_results):
    """
    Given the output from the plate recognition model (assumed to return detections 
    for individual characters with bounding boxes and class indices), this function 
    extracts and sorts them by x-coordinate and then maps the class indices to characters.
    """
    if not read_results:
        return ""
    
    # Use the first result (assuming one inference per crop)
    result = read_results[0]
    try:
        boxes = result.boxes.xyxy.cpu().numpy()  # shape: (N,4)
        classes = result.boxes.cls.cpu().numpy()   # shape: (N,)
    except Exception as e:
        print("Error processing recognition output:", e)
        return ""
    
    detections = []
    for box, cls in zip(boxes, classes):
        x1 = int(box[0])
        detections.append((x1, int(cls)))
    
    detections.sort(key=lambda x: x[0])
    plate_text = "".join(idx_to_char.get(cls, "") for _, cls in detections)
    return plate_text

# -----------------------------
# Main Processing Function
# -----------------------------
def process_image(image_path):
    """
    Processes a single image:
      1. Runs the plate detection model on the entire image.
      2. For each detected plate region, crops the region and runs the plate recognition model.
      3. Annotates the original image with bounding boxes and the recognized plate text.
    
    Returns the combined recognized text (if multiple boxes) and the annotated image.
    """
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Could not read", image_path)
        return "", None
    
    # Run detection on the entire image to find plates
    detection_results = plate_detector(image)
    predicted_texts = []
    
    # For every detected plate region
    for detection in detection_results:
        # Each detection can have multiple bounding boxes
        for box in detection.boxes.xyxy.cpu().numpy().astype(int):
            x1, y1, x2, y2 = box
            # Draw the detection box
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Crop the detected plate region
            crop = image[y1:y2, x1:x2]
            # Run the plate recognition model on the cropped plate region
            read_results = plate_reader(crop)
            pred_text = extract_plate_text(read_results)
            predicted_texts.append(pred_text)
            
            # Annotate recognized text above the bounding box
            cv2.putText(image, pred_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.9, (0, 255, 0), 2)
    
    combined_text = " ".join(predicted_texts)
    return combined_text, image

# -----------------------------
# Run Over Directory
# -----------------------------
def run_on_directory(test_dir, output_dir="recognized"):
    """
    Iterates through all JPG images in the test directory, runs the detection 
    and recognition pipeline, and saves annotated images to the output directory.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_files = glob.glob(os.path.join(test_dir, "*.png"))
    for image_path in image_files:
        filename = os.path.basename(image_path)
        pred_text, annotated_img = process_image(image_path)
        if annotated_img is not None:
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, annotated_img)
            print(f"{filename}: {pred_text}")

if __name__ == "__main__":
    test_dir = "./final_test"  # Directory with the 30 test images
    run_on_directory(test_dir, output_dir="./final_test_recognized")



0: 640x640 1 license_plate, 11.1ms
Speed: 1.8ms preprocess, 11.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 128x416 1 1, 2 2s, 2 6s, 1 8, 1 9, 1 ص, 12.8ms
Speed: 0.5ms preprocess, 12.8ms inference, 1.2ms postprocess per image at shape (1, 3, 128, 416)
Could not load the specified font. Please ensure the font file exists and supports Arabic.
day_00001.png: 66ص92218

0: 480x640 2 license_plates, 12.2ms
Speed: 1.6ms preprocess, 12.2ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 128x416 3 5s, 2 6s, 1 7, 1 8, 1 و, 11.6ms
Speed: 0.6ms preprocess, 11.6ms inference, 1.6ms postprocess per image at shape (1, 3, 128, 416)
Could not load the specified font. Please ensure the font file exists and supports Arabic.

0: 96x416 1 1, 2 2s, 1 4, 2 6s, 1 7, 1 ج, 12.6ms
Speed: 0.6ms preprocess, 12.6ms inference, 1.3ms postprocess per image at shape (1, 3, 96, 416)
Could not load the specified font. Please ensure the font file exists and supports Arabic.